这一节是讲CNN中另外一种重要的层，也就是pooling layer

实际上你可以发现，传统的卷积层对物体位置的变化是非常敏感的，看边缘检测的那个卷积核就能发现

为了解决这个位置变化的敏感性，我们引入了池化层，池化层实际上和卷积层的操作有点类似
1. 池化层是对卷积输出做的，因为卷积输出很敏感，所以用池化层一定程度上消除空间信息
2. 2d最大池化层：取窗口部分最大的像素
3. 池化层与卷积核不一样，完全没有可学习的参数
4. 与卷积层相似的，作用于多输入通道的时候，对每一个输入的通道我们都应用池化层来降低平移敏感性
5. 因为池化层对于每个通道我都会用池化层，所以池化层输入输出的通道数量始终是保持不变的（通道融合这个事情可以交给卷积层去做）
6. 平均池化层就是最大操作变成平均

其实这种计算方法我们直观上就能感觉到，确实带来了一定的平移稳定性

接下来我们来实现一个简单的池化层

In [55]:
import torch
from torch import nn
from d2l import torch as d2l

# 池化层的正向传播
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    # 输出大小的规律和卷积层是完全一模一样的
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [56]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [57]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

接下来我们用torch框架实现一下带有padding和stride的池化层

In [58]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同。 因此，如果我们使用形状为(3, 3)的汇聚窗口，那么默认情况下，我们得到的步幅形状为(3, 3)。

也就是每个窗口之间没有重叠

这个设定总感觉很抽象......

In [59]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [60]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [61]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

多通道输入输出的情况

In [62]:
X = torch.cat((X, X + 1), 1)
# 这里我们用的是cat函数，而没有用stack函数，这两个函数看起来相似，但是是有差别的
# stack函数在堆叠的时候会创建一个新的维度，在这个新建的维度上把几个对象堆叠起来，比如
# (3, 4) (3, 4) (3, 4)这三个矩阵堆叠，会变成一个(3, 3, 4)的3d矩阵，创建了一个新的维度
# cat则是按照你要求的维度堆叠，比如上面的例子，如果指定dim=1，新的矩阵维度会是(3, 12)
X
# X.shape # 像这里就是1244的维度结果，符合cat函数的运行方式

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [63]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

# 小结
* 对于给定输入元素，最大汇聚层会输出该窗口内的最大值，平均汇聚层会输出该窗口内的平均值。
* 汇聚层的主要优点之一是减轻卷积层对位置的过度敏感。
* 我们可以指定汇聚层的填充和步幅。
* 使用最大汇聚层以及大于1的步幅，可减少空间维度（如高度和宽度）。
* 汇聚层的输出通道数与输入通道数相同。

其实池化层的作用就是，让卷积层的输出的结果的空间信息“模糊化”